In [0]:
import numpy as np
import pandas as pd
import time
from sklearn.manifold import TSNE

import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('classic')
%matplotlib inline

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier

In [0]:
df = pd.read_csv('Training.csv')
df.head()

,Id,Product_Info_1,Product_Info_2,Product_Info_3,Product_Info_4,Product_Info_5,Product_Info_6,Product_Info_7,Ins_Age,Ht,Wt,BMI,Employment_Info_1,Employment_Info_2,Employment_Info_3,Employment_Info_4,Employment_Info_5,Employment_Info_6,InsuredInfo_1,InsuredInfo_2,InsuredInfo_3,InsuredInfo_4,InsuredInfo_5,InsuredInfo_6,InsuredInfo_7,Insurance_History_1,Insurance_History_2,Insurance_History_3,Insurance_History_4,Insurance_History_5,Insurance_History_7,Insurance_History_8,Insurance_History_9,Family_Hist_1,Family_Hist_2,Family_Hist_3,Family_Hist_4,Family_Hist_5,Medical_History_1,Medical_History_2,...,Medical_Keyword_10,Medical_Keyword_11,Medical_Keyword_12,Medical_Keyword_13,Medical_Keyword_14,Medical_Keyword_15,Medical_Keyword_16,Medical_Keyword_17,Medical_Keyword_18,Medical_Keyword_19,Medical_Keyword_20,Medical_Keyword_21,Medical_Keyword_22,Medical_Keyword_23,Medical_Keyword_24,Medical_Keyword_25,Medical_Keyword_26,Medical_Keyword_27,Medical_Keyword_28,Medical_Keyword_29,Medical_Keyword_30,Medical_Keyword_31,Medical_Keyword_32,Medical_Keyword_33,Medical_Keyword_34,Medical_Keyword_35,Medical_Keyword_36,Medical_Keyword_37,Medical_Keyword_38,Medical_Keyword_39,Medical_Keyword_40,Medical_Keyword_41,Medical_Keyword_42,Medical_Keyword_43,Medical_Keyword_44,Medical_Keyword_45,Medical_Keyword_46,Medical_Keyword_47,Medical_Keyword_48,Response
0,2,1,D3,10,0.076923,2,1,1,0.641791,0.581818,0.148536,0.323008,0.028,12,1,0.0,3,NaN,1,2,6,3,1,2,1,1,1,3,1,0.000667,1,1,2,2,NaN,0.598039,NaN,0.526786,4.0,112,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8
1,5,1,A1,26,0.076923,2,3,1,0.059701,0.600000,0.131799,0.272288,0.000,1,3,0.0,2,0.0018,1,2,6,3,1,2,1,2,1,3,1,0.000133,1,3,2,2,0.188406,NaN,0.084507,NaN,5.0,412,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4
2,6,1,E1,26,0.076923,2,3,1,0.029851,0.745455,0.288703,0.428780,0.030,9,1,0.0,2,0.0300,1,2,8,3,1,1,1,2,1,1,3,NaN,3,2,3,3,0.304348,NaN,0.225352,NaN,10.0,3,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8
3,7,1,D4,10,0.487179,2,3,1,0.164179,0.672727,0.205021,0.352438,0.042,9,1,0.0,3,0.2000,2,2,8,3,1,2,1,2,1,1,3,NaN,3,2,3,3,0.420290,NaN,0.352113,NaN,0.0,350,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8
4,8,1,D2,26,0.230769,2,3,1,0.417910,0.654545,0.234310,0.424046,0.027,9,1,0.0,2,0.0500,1,2,6,3,1,2,1,2,1,1,3,NaN,3,2,3,2,0.463768,NaN,0.408451,NaN,NaN,162,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8


In [0]:
df.describe()

,Id,Product_Info_1,Product_Info_3,Product_Info_4,Product_Info_5,Product_Info_6,Product_Info_7,Ins_Age,Ht,Wt,BMI,Employment_Info_1,Employment_Info_2,Employment_Info_3,Employment_Info_4,Employment_Info_5,Employment_Info_6,InsuredInfo_1,InsuredInfo_2,InsuredInfo_3,InsuredInfo_4,InsuredInfo_5,InsuredInfo_6,InsuredInfo_7,Insurance_History_1,Insurance_History_2,Insurance_History_3,Insurance_History_4,Insurance_History_5,Insurance_History_7,Insurance_History_8,Insurance_History_9,Family_Hist_1,Family_Hist_2,Family_Hist_3,Family_Hist_4,Family_Hist_5,Medical_History_1,Medical_History_2,Medical_History_3,...,Medical_Keyword_10,Medical_Keyword_11,Medical_Keyword_12,Medical_Keyword_13,Medical_Keyword_14,Medical_Keyword_15,Medical_Keyword_16,Medical_Keyword_17,Medical_Keyword_18,Medical_Keyword_19,Medical_Keyword_20,Medical_Keyword_21,Medical_Keyword_22,Medical_Keyword_23,Medical_Keyword_24,Medical_Keyword_25,Medical_Keyword_26,Medical_Keyword_27,Medical_Keyword_28,Medical_Keyword_29,Medical_Keyword_30,Medical_Keyword_31,Medical_Keyword_32,Medical_Keyword_33,Medical_Keyword_34,Medical_Keyword_35,Medical_Keyword_36,Medical_Keyword_37,Medical_Keyword_38,Medical_Keyword_39,Medical_Keyword_40,Medical_Keyword_41,Medical_Keyword_42,Medical_Keyword_43,Medical_Keyword_44,Medical_Keyword_45,Medical_Keyword_46,Medical_Keyword_47,Medical_Keyword_48,Response
count,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59362.000000,59381.000000,59381.000000,52602.000000,59381.000000,48527.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,33985.000000,59381.000000,59381.000000,59381.000000,59381.000000,30725.000000,25140.000000,40197.000000,17570.000000,50492.000000,59381.000000,59381.000000,...,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000
mean,39507.211515,1.026355,24.415655,0.328952,2.006955,2.673599,1.043583,0.405567,0.707283,0.292587,0.469462,0.077582,8.641821,1.300904,0.006283,2.142958,0.361469,1.209326,2.007427,5.835840,2.883666,1.027180,1.409188,1.038531,1.727606,1.055792,2.146983,1.958707,0.001733,1.901989,2.048484,2.419360,2.686230,0.474550,0.497737,0.444890,0.484635,7.962172,253.987100,2.102171,...,0.036459,0.058015,0.010003,0.005962,0.007848,0.190465,0.012715,0.009161,0.007494,0.009296,0.008134,0.014601,0.037167,0.097775,0.018895,0.089456,0.013439,0.011856,0.014937,0.011755,0.025042,0.010896,0.021168,0.022836,0.020646,0.006938,0.010407,0.066587,0.006837,0.013658,0.056954,0.010054,0.045536,0.010710,0.007528,0.013691,0.008488,0.019905,0.054496,5.636837
std,22815.883089,0.160191,5.072885,0.282562,0.083107,0.739103,0.291949,0.197190,0.074239,0.089037,0.122213,0.082347,4.227082,0.715034,0.032816,0.350033,0.349551,0.417939,0.085858,2.674536,0.320627,0.231566,0.491688,0.274915,0.445195,0.329328,0.989139,0.945739,0.007338,0.971223,0.755149,0.509577,0.483159,0.154959,0.140187,0.163012,0.129200,13.027697,178.621154,0.303098,...,0.187432,0.233774,0.099515,0.076981,0.088239,0.392671,0.112040,0.095275,0.086244,0.095967,0.089821,0.119949,0.189172,0.297013,0.136155,0.285404,0.115145,0.108237,0.121304,0.107780,0.156253,0.103813,0.143947,0.149380,0.142198,0.083007,0.101485,0.249307,0.082405,0.116066,0.231757,0.099764,0.208479,0.102937,0.086436,0.116207,0.091737,0.139676,0.226995,2.456833
min,2.000000,1.000000,1.000000,0.000000,2.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.

In [0]:
num_rows = len(df.index)
num_rows

59381

In [0]:
def get_columns_to_drop(df, missing_value_threshold):
  cols_to_drop = []
  for c in df.columns:
    if df[c].count() / num_rows < missing_value_threshold:
      print(f'# missing values in {c} is {num_rows - df[c].count()}')
      cols_to_drop.append(c)
  return cols_to_drop


In [0]:
# firstly lets take the theoritically acceptable rate i.e. 20-30%
cols_to_drop = get_columns_to_drop(df, 0.75)

# missing values in Insurance_History_5 is 25396
# missing values in Family_Hist_2 is 28656
# missing values in Family_Hist_3 is 34241
# missing values in Family_Hist_4 is 19184
# missing values in Family_Hist_5 is 41811
# missing values in Medical_History_10 is 58824
# missing values in Medical_History_15 is 44596
# missing values in Medical_History_24 is 55580
# missing values in Medical_History_32 is 58274


In [0]:
# Approach: these values wont help if not worsen our model
df = df.drop(columns=cols_to_drop)

In [0]:
# Finding rest of the missing values
def describe_missing_info(df):
  cnt = 0
  for c in df.columns:
    if df[c].count() < num_rows:
      print(f'# missing values in {c} is {num_rows - df[c].count()}')
      cnt += 1
  print(f'# columns that have no missing values : {len(df.columns) - cnt}')

In [0]:
describe_missing_info(df)

# missing values in Employment_Info_1 is 19
# missing values in Employment_Info_4 is 6779
# missing values in Employment_Info_6 is 10854
# missing values in Insurance_History_5 is 25396
# missing values in Family_Hist_2 is 28656
# missing values in Family_Hist_3 is 34241
# missing values in Family_Hist_4 is 19184
# missing values in Family_Hist_5 is 41811
# missing values in Medical_History_1 is 8889
# missing values in Medical_History_10 is 58824
# missing values in Medical_History_15 is 44596
# missing values in Medical_History_24 is 55580
# missing values in Medical_History_32 is 58274
# columns that have no missing values : 115


In [0]:
# dealing with categorical var first
# using label encoding ( as it was specified in mail )
catcol_str = 'Product_Info_1, Product_Info_2, Product_Info_3, Product_Info_5, Product_Info_6, Product_Info_7, Employment_Info_2, Employment_Info_3, Employment_Info_5, InsuredInfo_1, InsuredInfo_2, InsuredInfo_3, InsuredInfo_4, InsuredInfo_5, InsuredInfo_6, InsuredInfo_7, Insurance_History_1, Insurance_History_2, Insurance_History_3, Insurance_History_4, Insurance_History_7, Insurance_History_8, Insurance_History_9, Family_Hist_1, Medical_History_2, Medical_History_3, Medical_History_4, Medical_History_5, Medical_History_6, Medical_History_7, Medical_History_8, Medical_History_9, Medical_History_11, Medical_History_12, Medical_History_13, Medical_History_14, Medical_History_16, Medical_History_17, Medical_History_18, Medical_History_19, Medical_History_20, Medical_History_21, Medical_History_22, Medical_History_23, Medical_History_25, Medical_History_26, Medical_History_27, Medical_History_28, Medical_History_29, Medical_History_30, Medical_History_31, Medical_History_33, Medical_History_34, Medical_History_35, Medical_History_36, Medical_History_37, Medical_History_38, Medical_History_39, Medical_History_40, Medical_History_41'

object_columns = catcol_str.split(', ')
df[object_columns] = df[object_columns].apply(lambda x: x.astype('category'))
cat_columns = df.select_dtypes(['category']).columns
df[cat_columns] = df[cat_columns].apply(lambda x: x.cat.codes)
df[cat_columns]

,Product_Info_1,Product_Info_2,Product_Info_3,Product_Info_5,Product_Info_6,Product_Info_7,Employment_Info_2,Employment_Info_3,Employment_Info_5,InsuredInfo_1,InsuredInfo_2,InsuredInfo_3,InsuredInfo_4,InsuredInfo_5,InsuredInfo_6,InsuredInfo_7,Insurance_History_1,Insurance_History_2,Insurance_History_3,Insurance_History_4,Insurance_History_7,Insurance_History_8,Insurance_History_9,Family_Hist_1,Medical_History_2,Medical_History_3,Medical_History_4,Medical_History_5,Medical_History_6,Medical_History_7,Medical_History_8,Medical_History_9,Medical_History_11,Medical_History_12,Medical_History_13,Medical_History_14,Medical_History_16,Medical_History_17,Medical_History_18,Medical_History_19,Medical_History_20,Medical_History_21,Medical_History_22,Medical_History_23,Medical_History_25,Medical_History_26,Medical_History_27,Medical_History_28,Medical_History_29,Medical_History_30,Medical_History_31,Medical_History_33,Medical_History_34,Medical_History_35,Medical_History_36,Medical_History_37,Medical_History_38,Medical_History_39,Medical_History_40,Medical_History_41
0,0,16,8,0,0,0,10,0,1,0,0,5,1,0,1,0,0,0,2,0,0,0,1,1,102,1,0,0,2,1,1,0,2,1,2,2,2,2,0,0,1,0,1,2,0,2,2,0,2,1,2,0,2,0,1,1,0,2,2,2
1,0,0,22,0,1,0,0,1,0,0,0,5,1,0,1,0,1,0,2,0,0,2,1,1,368,1,0,0,2,1,1,0,2,1,2,2,0,2,0,0,1,0,1,2,0,2,2,0,2,1,2,1,0,0,1,1,0,2,2,0
2,0,18,22,0,1,0,7,0,0,0,0,7,1,0,0,0,1,0,0,2,2,1,2,2,2,1,1,0,2,1,1,1,2,1,2,2,0,2,0,0,1,0,1,2,1,1,2,0,2,1,2,1,2,0,2,1,0,2,2,0
3,0,17,8,0,1,0,7,0,1,1,0,7,1,0,1,0,1,0,0,2,2,1,2,2,312,1,1,0,2,1,1,1,2,1,2,2,0,2,0,0,1,1,1,2,0,2,2,0,2,1,2,1,2,0,1,1,0,2,2,0
4,0,15,22,0,1,0,7,0,0,0,0,5,1,0,1,0,1,0,0,2,2,1,2,1,148,1,1,0,2,1,1,1,2,1,2,2,0,2,0,0,1,0,1,2,1,1,2,0,2,1,2,1,2,0,2,1,0,2,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59376,0,14,8,0,1,0,0,1,1,0,0,7,1,0,1,0,1,0,0,2,2,1,2,2,237,1,0,0,2,1,1,1,2,1,2,2,0,2,0,0,1,0,1,2,0,2,2,0,2,1,2,1,2,0,1,1,0,2,2,2
59377,0,16,22,0,1,0,7,0,0,0,0,2,1,0,0,0,1,0,2,1,0,2,1,2,436,1,1,0,2,1,1,1,2,1,2,2,0,2,0,0,1,0,1,2,1,1,2,0,2,1,2,1,2,0,2,1,0,2,2,0
59378,0,18,22,0,1,0,7,0,0,0,0,5,1,0,0,0,1,0,0,2,2,1,2,2,148,1,1,0,2,1,1,1,2,1,2,2,0,2,0,0,1,0,1,2,1,1,2,0,2,1,2,1,0,0,2,1,0,2,2,0
59379,0,15,8,0,1,0,7,0,1,0,0,2,1,0,1,0,1,0,0,2,2,1,2,1,13,1,0,0,2,1,1,1,2,1,0,2,0,2,0,0,1,0,1,2,0,2,2,0,2,1,2,0,2,0,1,1,0,2,2,2


In [0]:
# as described Emp_hist_1,4,6 are cotinuous vars  &  med_hist_1 is a discrete var
# we will follow 2 approaches, firstly taking mean(c) & median(d) values and the second approach by building a model

cont_miss_val = ['Employment_Info_1', 'Employment_Info_4', 'Employment_Info_6']
disc_miss_val = ['Medical_History_1']

df[cont_miss_val] = df[cont_miss_val].fillna(value=df[cont_miss_val].mean())
df[disc_miss_val] = df[disc_miss_val].fillna(value=df[disc_miss_val].mode()['Medical_History_1'][0])


In [0]:
# As the target class is ordinal, 
# using a classifier would lose the ordering information of the variable
# using a regressor wouldn't be helpful as there is no interval/ratio correspondence given
# eg. class 1 : Lower Class   : Rs 5k/-
#     class 2 : Middle Class  : Rs 5-600k
#     class 3 : Higher Class  : Rs 600k/+
# here the classification is not like red, blue, green; it has some value, each depicting something 
# and here regression can be performed as we can find the income and choose the class
# but in our case, we dont have this interval/ratio correspodence.

In [0]:
y = df.Response
X = df.drop(axis=1, columns=['Response'])

In [0]:
y.unique()

array([8, 4, 1, 6, 2, 7, 3, 5])

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [0]:
# Model 1 : Using a classifier

# Model 2 : Using a regressor

In [0]:
from sklearn.neighbors import KNeighborsClassifier

for i in range(500, 501, 4): # even steps so as to get an odd n, each time
  clf = KNeighborsClassifier(n_neighbors=i)
  clf.fit(X_train, y_train)
  print(i, accuracy_score(y_test, clf.predict(X_test)))

# starting from 5 to 49 gave 20-30% accuracy
# starting from 41 to 97 gave 29-31% accuracy
# going around 150-200, 400, 500 + gives 32%

500 0.32843863666981005


In [0]:
# normalizing the data

from sklearn.preprocessing import normalize

normalized_X = normalize(X)
norm_X_train, norm_X_test = train_test_split(normalized_X)
for i in range(5, 100, 4): # even steps so as to get an odd n, each time
  clf = KNeighborsClassifier(n_neighbors=i)
  clf.fit(norm_X_train, y_train)
  print(i, accuracy_score(y_test, clf.predict(norm_X_test)))
# going for 5-100 neighbors also gave around 20-31% accuracy score

5 0.2031523642732049
9 0.22659302168934392
13 0.24841708204230095
17 0.26209079886838205
21 0.27488885895190623
25 0.2814899636265661
29 0.29051596389599893
33 0.2983295163680453
37 0.3023710090260003
41 0.30600835241815977
45 0.30742287484844405
49 0.31119493466253534
53 0.3132830392024788
57 0.3160447258520814
61 0.31840226323588844
65 0.32156809915128653
69 0.32197224841708205
73 0.32331941263640035
77 0.3246665768557187
81 0.3237909201131618
85 0.3259463828640711
89 0.3270241142395258
93 0.32763033813821907
97 0.3271588306614576


In [0]:
# Building Decision trees, with restriction to its depth

In [0]:
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
print(accuracy_score(y_test, clf.predict(X_test)))

0.39606628047959047


In [0]:
for i in range(5, 50, 5):
  clf = DecisionTreeClassifier(max_depth=i)
  clf.fit(X_train, y_train)
  print(i, accuracy_score(y_test, clf.predict(X_test)))

5 0.49939377610130675
10 0.519399164758184
15 0.4911760743634649
20 0.4529166105348242
25 0.4316987740805604
30 0.41499393776101307
35 0.40953792267277384
40 0.4030715344200458
45 0.3985585342853294


In [0]:
for i in range(2, 14):
  clf = DecisionTreeClassifier(max_depth=i)
  clf.fit(X_train, y_train)
  print(i, accuracy_score(y_test, clf.predict(X_test)))

2 0.42604068435942344
3 0.4576990435134043
4 0.46935201401050786
5 0.49939377610130675
6 0.5104405226997171
7 0.5175131348511384
8 0.5188602990704567
9 0.5186582244375589
10 0.5187929408594908
11 0.5126633436615924
12 0.5084871345817055
13 0.5022901791728411


In [0]:
# Building a Random Forest 

In [0]:
# no max depth restriction
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
accuracy_score(y_test, clf.predict(X_test))

0.5311194934662535

In [0]:
# Restricting depth to 7
clf = RandomForestClassifier(max_depth=7)
clf.fit(X_train, y_train)
accuracy_score(y_test, clf.predict(X_test))

0.48565270106425973

In [0]:
clf =MLPClassifier()
clf.fit(X_train, y_train)
print(accuracy_score(y_test, clf.predict(X_test)))

0.3105887107638421


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [0]:
# A neural network way

In [0]:
clf =MLPClassifier(hidden_layer_sizes=(64, ))
clf.fit(X_train, y_train)
print(accuracy_score(y_test, clf.predict(X_test)))

0.34615384615384615


In [0]:
# Model : calssifier that finds probability for each class
# This is a model that I saw on Medium, and found very interesting, and it handles ordinal classification

In [0]:
from sklearn.base import clone

# considering classes from 1 to k

class OrdinalClassifier():
    
    def __init__(self, clf):
        self.clf = clf
        self.clfs = {}
    
    def fit(self, X, y):
        self.unique_class = sorted(y.unique())
        if len(self.unique_class) > 2:
            for i in self.unique_class[:-1]:
                # for each k - 1 ordinal value we fit a binary classification problem
                binary_y = (y > i).astype(np.uint8)
                clf = clone(self.clf)
                clf.fit(X, binary_y)
                self.clfs[i] = clf
    
    def predict_proba(self, X):
        clfs_predict = {k:self.clfs[k].predict_proba(X) for k in self.clfs.keys()}
        # return(clfs_predict)
        predicted = []
        # V1 = 1 - Pr(y > V1)
        predicted.append(clfs_predict[self.unique_class[0]][:,1])
        for i in range(1, len(self.unique_class)-1):
            # Vi = Pr(y > Vi-1) - Pr(y > Vi)
            predicted.append(clfs_predict[self.unique_class[i]-1][:,1] - clfs_predict[self.unique_class[i]][:,1])
        # Vk = Pr(y > Vk-1)
        predicted.append(clfs_predict[self.unique_class[i]][:,1])
        return np.vstack(predicted).T
    
    def predict(self, X):        
        # return self.predict_proba(X)
        return np.argmax(self.predict_proba(X), axis=1)

In [0]:
from sklearn.tree import DecisionTreeClassifier
for i in range(-5,9):
  clf = OrdinalClassifier(DecisionTreeClassifier(max_depth=30+i))
  clf.fit(X_train, y_train)
  model3_pred = clf.predict(X_test)
  model3_pred = [i+1 for i in model3_pred]
  print(i+30, accuracy_score(y_test, model3_pred))

25 0.20207463289775024
26 0.20510575239121648
27 0.2136602451838879
28 0.20604876734473931
29 0.21628721541155868
30 0.21904890206116126
31 0.2164892900444564
32 0.2092146032601374
33 0.1970901252862724
34 0.18860299070456688
35 0.19567560285598815
36 0.18557187121110064
37 0.18159773676411153
38 0.17762360231712246


In [0]:
# not providing max_depth results into probability being 0 or 1 and so we need to have a max_depth value so as not to overfit
# max depth :  accuracy
# no 0.0969284655799542
# 10-70 :  10,15,21,17,10,10
# 25-38 :  19-21-19

# At 33 depth, we get the best accuracy, being 21.53%, which is actually very very poor


In [0]:
# Builing a LogReg model for multiclass claasification

In [0]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

scaler = StandardScaler()
scaler.fit(X_train)
scX_train = scaler.transform(X_train)
scX_test = scaler.transform(X_test)

clf = LogisticRegression(solver='saga', max_iter=800)
clf.fit(scX_train, y_train)
print(accuracy_score(y_test, clf.predict(scX_test)))

0.4948807759665903


In [0]:
# Now we generate values for missing ones with model
# lets just repeat the process

df = pd.read_csv('Training.csv')

num_rows = len(df.index)

cols_to_drop = get_columns_to_drop(df, 0.75)
print('-----')
df = df.drop(columns=cols_to_drop)

describe_missing_info(df)

catcol_str = 'Product_Info_1, Product_Info_2, Product_Info_3, Product_Info_5, Product_Info_6, Product_Info_7, Employment_Info_2, Employment_Info_3, Employment_Info_5, InsuredInfo_1, InsuredInfo_2, InsuredInfo_3, InsuredInfo_4, InsuredInfo_5, InsuredInfo_6, InsuredInfo_7, Insurance_History_1, Insurance_History_2, Insurance_History_3, Insurance_History_4, Insurance_History_7, Insurance_History_8, Insurance_History_9, Family_Hist_1, Medical_History_2, Medical_History_3, Medical_History_4, Medical_History_5, Medical_History_6, Medical_History_7, Medical_History_8, Medical_History_9, Medical_History_11, Medical_History_12, Medical_History_13, Medical_History_14, Medical_History_16, Medical_History_17, Medical_History_18, Medical_History_19, Medical_History_20, Medical_History_21, Medical_History_22, Medical_History_23, Medical_History_25, Medical_History_26, Medical_History_27, Medical_History_28, Medical_History_29, Medical_History_30, Medical_History_31, Medical_History_33, Medical_History_34, Medical_History_35, Medical_History_36, Medical_History_37, Medical_History_38, Medical_History_39, Medical_History_40, Medical_History_41'

object_columns = catcol_str.split(', ')
df[object_columns] = df[object_columns].apply(lambda x: x.astype('category'))
cat_columns = df.select_dtypes(['category']).columns
df[cat_columns] = df[cat_columns].apply(lambda x: x.cat.codes)



# missing values in Insurance_History_5 is 25396
# missing values in Family_Hist_2 is 28656
# missing values in Family_Hist_3 is 34241
# missing values in Family_Hist_4 is 19184
# missing values in Family_Hist_5 is 41811
# missing values in Medical_History_10 is 58824
# missing values in Medical_History_15 is 44596
# missing values in Medical_History_24 is 55580
# missing values in Medical_History_32 is 58274
-----
# missing values in Employment_Info_1 is 19
# missing values in Employment_Info_4 is 6779
# missing values in Employment_Info_6 is 10854
# missing values in Medical_History_1 is 8889
# columns that have no missing values : 115


In [0]:

cont_miss_val = ['Employment_Info_1', 'Employment_Info_4', 'Employment_Info_6']
disc_miss_val = ['Medical_History_1']


In [0]:
df_cols = df.columns
df = pd.DataFrame(disc_cols)
df.columns = df_cols
df['Medical_History_1'].isnull().sum()

0

In [0]:
aaa = set()
for i in df[[disc_miss_val[0]]].values:
  try:
    aaa.add(int(i[0]))
  except:
    pass
print(aaa)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 130, 131, 132, 134, 136, 138, 141, 143, 145, 146, 147, 148, 150, 153, 154, 155, 156, 158, 159, 160, 161, 162, 169, 171, 172, 173, 175, 176, 178, 179, 180, 182, 185, 187, 191, 193, 201, 223, 228, 229, 239, 240}


In [0]:
import sys
try:
  from impyute.imputation.cs import mice
except:
  !pip install impyute
  from impyute.imputation.cs import mice

df_cols = df.columns
df = pd.DataFrame(mice(df))
df.columns = df_cols

/usr/local/lib/python3.6/dist-packages/impyute/util/preprocess.py:52: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  args[0] = args[0].as_matrix()


In [0]:
df[disc_miss_val] = df[disc_miss_val].astype(int)

In [0]:
y = df.Response
X = df.drop(axis=1, columns=['Response'])

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [0]:
# Building a KNN model

In [0]:
from sklearn.neighbors import KNeighborsClassifier

for i in range(5, 50, 4): # even steps so as to get an odd n, each time
  clf = KNeighborsClassifier(n_neighbors=i)
  clf.fit(X_train, y_train)
  print(i, accuracy_score(y_test, clf.predict(X_test)))

# starting from 5 to 49 gave 21-30% accuracy

5 0.2139970362387175
9 0.23716826081099285
13 0.25488347029502895
17 0.2670753064798599
21 0.2691634110198033
25 0.278324127711168
29 0.2839148592213391
33 0.2888993668328169
37 0.2952310386636131
41 0.2979927253132157
45 0.29981139700929543
49 0.30338138219048905


In [0]:
# normalizing the data

from sklearn.preprocessing import normalize

normalized_X = normalize(X)
norm_X_train, norm_X_test = train_test_split(normalized_X)
for i in range(5, 100, 4): # even steps so as to get an odd n, each time
  clf = KNeighborsClassifier(n_neighbors=i)
  clf.fit(norm_X_train, y_train)
  print(i, accuracy_score(y_test, clf.predict(norm_X_test)))
# going for 5-100 neighbors also gave around 20-31% accuracy score

5 0.20645291661053483
9 0.22787282769769635
13 0.2445776640172437
17 0.25643270914724503
21 0.2695675602855988
25 0.28135524720463423
29 0.2857335309174188
33 0.29092011316179445
37 0.2957025461403745
41 0.29651084467196553
45 0.2983295163680453
49 0.30297723292469353
53 0.30459382998787554
57 0.30742287484844405
61 0.309039471911626
65 0.31072342718577395
69 0.3118011585612286
73 0.3133503974134447
77 0.31442812878889936
81 0.31442812878889936
85 0.31476491984372895
89 0.31631415869594504
93 0.3161120840630473
97 0.3159100094301495


In [0]:
# BUilding a Decision Tree Classifier

In [0]:
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
print(accuracy_score(y_test, clf.predict(X_test)))

0.40132022093493197


In [0]:
for i in range(5, 50, 5):
  clf = DecisionTreeClassifier(max_depth=i)
  clf.fit(X_train, y_train)
  print(i, accuracy_score(y_test, clf.predict(X_test)))

5 0.49744038798329515
10 0.5150208810453994
15 0.4882123130809646
20 0.45951771520948403
25 0.433652162198572
30 0.4183618483093089
35 0.40495756432709146
40 0.3981543850195339
45 0.3959315640576586


In [0]:
for i in range(2, 14):
  clf = DecisionTreeClassifier(max_depth=i)
  clf.fit(X_train, y_train)
  print(i, accuracy_score(y_test, clf.predict(X_test)))

2 0.43392159504243566
3 0.4540617001212448
4 0.46881314832278054
5 0.49744038798329515
6 0.506533746463694
7 0.5111141048093762
8 0.5158291795769905
9 0.5163006870537519
10 0.5150882392563654
11 0.5101710898558535
12 0.5051192240334097
13 0.49905698504647716


In [0]:
# Building a neural network 

In [0]:
clf =MLPClassifier(hidden_layer_sizes=(128, 64, 8, ))
clf.fit(X_train, y_train)
print(accuracy_score(y_test, clf.predict(X_test)))

0.3222416812609457


In [0]:
# Building a Random Forest Approach

In [0]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()
clf.fit(X_train, y_train)
accuracy_score(y_test, clf.predict(X_test))

0.5259329112218779

In [0]:
# Trying a LogReg

In [0]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

scaler = StandardScaler()
scaler.fit(X_train)
scX_train = scaler.transform(X_train)
scX_test = scaler.transform(X_test)

clf = LogisticRegression(solver='saga', max_iter=800)
clf.fit(scX_train, y_train)
print(accuracy_score(y_test, clf.predict(scX_test)))

0.4803314023979523


In [0]:
# now at last using a dimensionality reduction to give 2d input to KNN

In [0]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
pca.fit(X)
op = pca.transform(X)

In [0]:
X2_train, X2_test = train_test_split(X, test_size=0.25)

In [0]:
clf = KNeighborsClassifier(n_neighbors=33)
clf.fit(X2_train, y_train)
accuracy_score(y_test, clf.predict(X2_test))

0.2965782028829314